In [1]:
import pandas as pd

## Carga del archivo tipo .xlsx y convertimos cada hoja en un archivo tipo .csv

In [2]:
#carga el archivo excel indicándole que lea todas las hojas
df_excel = pd.read_excel('datasets/homicidios.xlsx', sheet_name=None)

#indica la ruta y carpeta de los dataset del proyecto
carpeta_archivos = 'datasets/'

#itera sobre el diccionario 'df_excel', donde las claves con los nombres de las hojas de excel y los valores son los Dataframe respectivos     
for hoja_excel, df in df_excel.items():
    nombre_csv = f'{carpeta_archivos}{hoja_excel}.csv'
    df.to_csv(nombre_csv, index = False)
    print(f'Archivo csv {nombre_csv} creado exitosamente')   


Archivo csv datasets/HECHOS.csv creado exitosamente
Archivo csv datasets/DICCIONARIO_HECHOS.csv creado exitosamente
Archivo csv datasets/VICTIMAS.csv creado exitosamente
Archivo csv datasets/DICCIONARIO_VICTIMAS.csv creado exitosamente
Archivo csv datasets/clas.csv creado exitosamente


### Cargamos los archivos de trabajo 'Hechos' y 'Victimas' para revisión de información

In [3]:
df_hechos = pd.read_csv('datasets/Hechos.csv')                     #cargamos el archivo Hechos.csv para el análisis

In [4]:
df_victimas = pd.read_csv('datasets/VICTIMAS.csv')                 #cargamos el archivo VICTIMAS.csv para el análisis

## ETL SOBRE EL ARCHIVO 'Hechos'

### Revisión de la información y tipo de campos

In [63]:
df_hechos.head(4)                                                  #damos una mirada general a la información del archivo

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD


In [64]:
df_hechos.info()                                                       #revisamos las columnas y tipo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     696 non-null    object 
 1   N_VICTIMAS             696 non-null    int64  
 2   FECHA                  696 non-null    object 
 3   AAAA                   696 non-null    int64  
 4   MM                     696 non-null    int64  
 5   DD                     696 non-null    int64  
 6   HORA                   696 non-null    object 
 7   HH                     696 non-null    object 
 8   LUGAR_DEL_HECHO        696 non-null    object 
 9   TIPO_DE_CALLE          696 non-null    object 
 10  Calle                  695 non-null    object 
 11  Altura                 129 non-null    float64
 12  Cruce                  525 non-null    object 
 13  Dirección Normalizada  688 non-null    object 
 14  COMUNA                 696 non-null    int64  
 15  XY (CA

### Eliminamos los registros con valores nulos e identificados en campo 'ID'

In [5]:
df_hechos = df_hechos.dropna(subset = ['ID'])                   #eliminamos las filas vacias en el campo ID

### Aseguramos consistencia de datos en 'N_VICTIMAS' y 'COMUNA' pasando los datos NaN a 0

In [6]:
df_hechos['COMUNA'] = df_hechos['COMUNA'].fillna(0)
df_hechos['N_VICTIMAS'] = df_hechos['N_VICTIMAS'].fillna(0)

### Eliminamos las columnas que no se requieren

In [7]:
eliminar_columnas = ['AAAA', 'MM', 'DD', 'HORA', 'XY (CABA)']
df_hechos2 = df_hechos.drop(eliminar_columnas, axis = 1)

In [70]:
df_hechos2.columns                          #Revisamos las columnas resultantes

Index(['ID', 'N_VICTIMAS', 'FECHA', 'HH', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE',
       'Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'COMUNA', 'pos x',
       'pos y', 'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [8]:
df_hechos2['ID'].head(4)                        #verifico la información del campo 'ID' para posterior uninón con el dataset 'VICTIMAS'

0    2016-0001
1    2016-0002
2    2016-0003
3    2016-0004
Name: ID, dtype: object

## ETL sobre el archivo 'VICTIMAS'

In [9]:
df_victimas.head(4)                             #Damos una visual a la información del dataset

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD


In [10]:
df_victimas.info()                      #Revisamos la cantidad y tipo de columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_hecho             717 non-null    object
 1   FECHA                717 non-null    object
 2   AAAA                 717 non-null    int64 
 3   MM                   717 non-null    int64 
 4   DD                   717 non-null    int64 
 5   ROL                  717 non-null    object
 6   VICTIMA              717 non-null    object
 7   SEXO                 717 non-null    object
 8   EDAD                 717 non-null    object
 9   FECHA_FALLECIMIENTO  717 non-null    object
dtypes: int64(3), object(7)
memory usage: 56.1+ KB


### Elimino los registros nulos e identificados en campo 'ID_hecho'

In [11]:
df_victimas = df_victimas.dropna(subset = ['ID_hecho'])                   #eliminamos las filas vacias en el campo 'ID_hecho'

### Eliminamos columnas que no se requieren

In [13]:
eliminar_columnas = ['AAAA', 'MM', 'DD']
df_victimas2 = df_victimas.drop(eliminar_columnas, axis = 1)

In [76]:
df_victimas2.columns                    #Reviso columnas restantes

Index(['ID_hecho', 'FECHA', 'ROL', 'VICTIMA', 'SEXO', 'EDAD',
       'FECHA_FALLECIMIENTO'],
      dtype='object')

### Renombramos la columna 'ID_hecho' por 'ID'

In [14]:
df_victimas2 = df_victimas2.rename(columns={'ID_hecho': 'ID'})

In [15]:
df_victimas2.head(4)                        #Reviso la información después del cambio

,ID,FECHA,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18,SD


## Uno los dos dataframe 'Hechos' y 'Victimas' mediante el campo 'ID'

In [16]:
df_union = pd.merge(df_hechos2, df_victimas2, on = 'ID', how = 'inner')     #El merge es sobre el campo 'ID'

In [17]:
df_union.head(4)                    #Reviso la información del nuevo Dataframe

,ID,N_VICTIMAS,FECHA_x,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,...,pos y,PARTICIPANTES,VICTIMA_x,ACUSADO,FECHA_y,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",...,-34.68757022,MOTO-AUTO,MOTO,AUTO,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",...,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,...,-34.63189362,MOTO-AUTO,MOTO,AUTO,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",...,-34.68092974,MOTO-SD,MOTO,SD,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18,SD


In [18]:
df_union.info()                             #Reviso la nueva estructura de las columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     717 non-null    object 
 1   N_VICTIMAS             717 non-null    int64  
 2   FECHA_x                717 non-null    object 
 3   HH                     717 non-null    object 
 4   LUGAR_DEL_HECHO        717 non-null    object 
 5   TIPO_DE_CALLE          717 non-null    object 
 6   Calle                  716 non-null    object 
 7   Altura                 133 non-null    float64
 8   Cruce                  540 non-null    object 
 9   Dirección Normalizada  708 non-null    object 
 10  COMUNA                 717 non-null    int64  
 11  pos x                  717 non-null    object 
 12  pos y                  717 non-null    object 
 13  PARTICIPANTES          717 non-null    object 
 14  VICTIMA_x              717 non-null    object 
 15  ACUSAD

### Eliminamos las columnas duplicadas

In [20]:
eliminar_columnas = ['FECHA_y', 'VICTIMA_y']
df_union = df_union.drop(eliminar_columnas, axis = 1)

In [21]:
df_union.columns                                #Revisamos las columnas finales

Index(['ID', 'N_VICTIMAS', 'FECHA_x', 'HH', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE',
       'Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'COMUNA', 'pos x',
       'pos y', 'PARTICIPANTES', 'VICTIMA_x', 'ACUSADO', 'ROL', 'SEXO', 'EDAD',
       'FECHA_FALLECIMIENTO'],
      dtype='object')

#### Revisamos registros duplicados de la columna 'ID'

In [22]:
num_registros_duplicados = df_union.duplicated(subset=['ID']).sum()                     #Filtramos los registros duplicados para saber la cantidad
print("Número de registros duplicados en la columna 'ID':", num_registros_duplicados)   #Mostramos los registros duplicados

Número de registros duplicados en la columna 'ID': 21


In [23]:
df_union.shape                                                                          #Reviso el frame

(717, 20)

### Elimino registros duplicados en la columna 'ID'

In [24]:
df_union.drop_duplicates(subset=['ID'], keep='first', inplace=True)     #elimino registros duplicados dentro del dataframe original

In [25]:
df_union.shape                                                          #Reviso nueva estructura del frame

(696, 20)

### Reviso el tipo de dato de la columna 'VICTIMA_X'

In [26]:
valores_unicos_victima_x = df_union['VICTIMA_x'].unique()
print(valores_unicos_victima_x)

['MOTO' 'AUTO' 'PEATON' 'SD' 'CARGAS' 'BICICLETA' 'PASAJEROS' 'MOVIL'
 'OBJETO FIJO' 'PEATON_MOTO']


### Estadarizo los valores únicos de la columna 'VICTIMA_x'

In [27]:
df_union['VICTIMA_x'] = df_union['VICTIMA_x'].replace(['PEATON_MOTO'], 'SD')
df_union['VICTIMA_x'] = df_union['VICTIMA_x'].replace(['OBJETO FIJO'], 'SD')

### Verifico el tipo de valores de la columna 'EDAD'

In [28]:
valores_unicos_victima_x = df_union['EDAD'].unique()            #Reviso los valores de la columna 'EDAD'
print(valores_unicos_victima_x)

['19' '70' '30' '18' '29' '22' '16' '59' '65' '34' '41' '50' '38' '21'
 '52' '36' '20' '54' '56' '24' 'SD' '78' '79' '26' '57' '37' '58' '23'
 '60' '42' '53' '51' '40' '87' '76' '75' '35' '80' '43' '45' '67' '27'
 '55' '49' '81' '25' '33' '46' '83' '39' '28' '7' '48' '4' '82' '32' '17'
 '47' '61' '10' '95' '73' '84' '66' '85' '1' '15' '13' '77' '44' '31' '62'
 '74' '71' '11' '86' '69' '72' '12' '63' '92' '68' '91' '64' '5' '88']


#### Reemplazo los valores erróneos de la columna 'EDAD'

In [29]:
df_union['EDAD'] = df_union['EDAD'].replace(['SD'], '')

### Verifico el tipo de valores de la columna 'ACUSADO'

In [30]:
valores_unicos_victima_x = df_union['ACUSADO'].unique()             #Verifico los datos de la columna 'ACUSADO'
print(valores_unicos_victima_x)

['AUTO' 'PASAJEROS' 'SD' 'OBJETO FIJO' 'CARGAS' 'MOTO' 'MULTIPLE' 'OTRO'
 'BICICLETA' 'TREN']


#### Reemplazo los valores erróneos de la columna 'ACUSADO'

In [31]:
df_union['ACUSADO'] = df_union['ACUSADO'].replace(['OBJETO FIJO'], 'SD')

### Verifico datos faltantes

In [32]:
df_union.isna().sum()

ID                         0
N_VICTIMAS                 0
FECHA_x                    0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA_x                  0
ACUSADO                    0
ROL                        0
SEXO                       0
EDAD                       0
FECHA_FALLECIMIENTO        0
dtype: int64

#### La columna 'Altura' tiene el 82% de los datos faltantes por lo que se procede a eliminarla

In [33]:
df_union = df_union.drop(columns=['Altura'])                    # Eliminamos la columna 'Altura' 

### Verifico datos y columnas del Dataframe df_union

In [35]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
Index: 696 entries, 0 to 716
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     696 non-null    object
 1   N_VICTIMAS             696 non-null    int64 
 2   FECHA_x                696 non-null    object
 3   HH                     696 non-null    object
 4   LUGAR_DEL_HECHO        696 non-null    object
 5   TIPO_DE_CALLE          696 non-null    object
 6   Calle                  695 non-null    object
 7   Cruce                  525 non-null    object
 8   Dirección Normalizada  688 non-null    object
 9   COMUNA                 696 non-null    int64 
 10  pos x                  696 non-null    object
 11  pos y                  696 non-null    object
 12  PARTICIPANTES          696 non-null    object
 13  VICTIMA_x              696 non-null    object
 14  ACUSADO                696 non-null    object
 15  ROL                    696 n

In [36]:
df_union.isna().sum()

ID                         0
N_VICTIMAS                 0
FECHA_x                    0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA_x                  0
ACUSADO                    0
ROL                        0
SEXO                       0
EDAD                       0
FECHA_FALLECIMIENTO        0
dtype: int64

#### Revisamos el registro nulo en 'Calle' 

In [37]:
registro_calle_nula = df_union[df_union['Calle'].isnull()]          # Muestra el registro donde 'Calle' es nulo
print("Registro donde 'Calle' es nulo:")
registro_calle_nula

Registro donde 'Calle' es nulo:


,ID,N_VICTIMAS,FECHA_x,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA_x,ACUSADO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
121,2016-0151,1,2016-11-18,20,SD,CALLE,NaN,NaN,NaN,0,.,.,PEATON-SD,PEATON,SD,PEATON,SD,,SD


#### Para el registro nulo en campo 'Calle' vemos que falta mucha información en general, por lo que decidimos anular dicho registro

In [38]:
df_union.dropna(subset=['Calle'], inplace=True)                 # Eliminar el registro donde 'Calle' es nulo

In [39]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695 entries, 0 to 716
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     695 non-null    object
 1   N_VICTIMAS             695 non-null    int64 
 2   FECHA_x                695 non-null    object
 3   HH                     695 non-null    object
 4   LUGAR_DEL_HECHO        695 non-null    object
 5   TIPO_DE_CALLE          695 non-null    object
 6   Calle                  695 non-null    object
 7   Cruce                  525 non-null    object
 8   Dirección Normalizada  688 non-null    object
 9   COMUNA                 695 non-null    int64 
 10  pos x                  695 non-null    object
 11  pos y                  695 non-null    object
 12  PARTICIPANTES          695 non-null    object
 13  VICTIMA_x              695 non-null    object
 14  ACUSADO                695 non-null    object
 15  ROL                    695 n

### Cambio de nombre las columnas 'pos x' y 'pos y'

In [42]:
df_union = df_union.rename(columns={'pos x': 'Longitud', 'pos y': 'Latitud'})       #Cambio los nombres por Longitud y Latitud

In [44]:
dfx = df_union

In [45]:
dfx.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695 entries, 0 to 716
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     695 non-null    object
 1   N_VICTIMAS             695 non-null    int64 
 2   FECHA_x                695 non-null    object
 3   HH                     695 non-null    object
 4   LUGAR_DEL_HECHO        695 non-null    object
 5   TIPO_DE_CALLE          695 non-null    object
 6   Calle                  695 non-null    object
 7   Cruce                  525 non-null    object
 8   Dirección Normalizada  688 non-null    object
 9   COMUNA                 695 non-null    int64 
 10  Longitud               695 non-null    object
 11  Latitud                695 non-null    object
 12  PARTICIPANTES          695 non-null    object
 13  VICTIMA_x              695 non-null    object
 14  ACUSADO                695 non-null    object
 15  ROL                    695 n

## Guardo el dataframe final en un archivo .csv

In [51]:
df_union.to_csv('datasets/hechos_victimas_depurados.csv', index = False)